#  Load Karate Club

In [1]:
from scipy.linalg import fractional_matrix_power
import numpy as np
def power(matrix, fraction):
    return np.matrix(fractional_matrix_power(matrix, fraction))

In [2]:
from networkx import to_numpy_matrix
from src.data.datasets import load_karate_club
zkc = load_karate_club()

In [3]:
%time
from src.models.gcn_layers import GraphConv, SimpleGraphConv as GraphConvSkip
from mxnet.gluon.nn import HybridSequential
import numpy as np
from mxnet.ndarray import array, tanh, sigmoid
from mxnet.symbol import tanh, sigmoid
from mxnet.initializer import Xavier
from mxnet.ndarray import sigmoid, tanh
from mxnet.gluon.loss import SigmoidBinaryCrossEntropyLoss
A = to_numpy_matrix(zkc.meta_data['graph'])
D = np.array(np.sum(A, axis=0))[0]
D = np.matrix(np.diag(D))
#A = D**-1 * A
X = I = np.eye(A.shape[0])
degrees = np.array([[d] for d in np.array(np.sum(A, axis=0))[0]])
X = np.concatenate((I, degrees), axis=1)

A_hat = power(D, -0.5) * A * power(D, -0.5)
A_hat = array(A)
print(A_hat.shape)
X = array(I)
print(X.shape)
model = HybridSequential()
with model.name_scope():
    H_1 = GraphConvSkip(A_hat, in_units=X.shape[1], out_units=16, activation=tanh)
    model.add(H_1)
    H_2 = GraphConvSkip(A_hat, in_units=H_1.out_units, out_units=8, activation=tanh)
    model.add(H_2)

    H_3 = GraphConvSkip(A_hat, in_units=H_2.out_units, out_units=4, activation=tanh)
    model.add(H_3)
    
    model.add(
         GraphConvSkip(A_hat, in_units=H_3.out_units, out_units=1, activation=sigmoid)
    )
    
#model.hybridize()
model.initialize(Xavier(rnd_type='gaussian', magnitude=1))
preds = model(X)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs
(34, 34)
(34, 34)


In [6]:
%time
from mxnet import autograd
from mxnet.gluon import Trainer
from mxnet.ndarray import sum as ndsum
from sklearn.metrics import classification_report
import numpy as np

X_train = zkc.X_train.flatten()
y_train = zkc.y_train
X_test = zkc.X_test.flatten()
y_test = zkc.y_test

def train_and_test(model, X_train, y_train, X_test, y_test):
    epochs = 10000
    loss_sequence = []
    cross_entropy = SigmoidBinaryCrossEntropyLoss(from_sigmoid=True)
    trainer = Trainer(model.collect_params(), 'sgd', {'learning_rate': 0.001})

    for e in range(epochs):
        e += 1
        with autograd.record():
            preds = model(X)[X_train]
            loss = ndsum(cross_entropy(preds, array(zkc.y_train)))

        loss.backward()
        trainer.step(2)
        if (e % (epochs//10)) == 0:
            print("Epoch %s, loss: %s" % (e, loss.asscalar()))


    preds = model(X)[X_test].asnumpy().flatten()
    preds = np.where(preds >= 0.5, 1, 0)
    print(classification_report(preds, y_test))

train_and_test(model, X_train, y_train, X_test, y_test)

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 10 µs
Epoch 1000, loss: 0.9978477
Epoch 2000, loss: 0.747853
Epoch 3000, loss: 0.5399451
Epoch 4000, loss: 0.39453673
Epoch 5000, loss: 0.29920155
Epoch 6000, loss: 0.23599176
Epoch 7000, loss: 0.19247495
Epoch 8000, loss: 0.16127175
Epoch 9000, loss: 0.13806579
Epoch 10000, loss: 0.12026313
              precision    recall  f1-score   support

           0       0.75      0.71      0.73        17
           1       0.69      0.73      0.71        15

   micro avg       0.72      0.72      0.72        32
   macro avg       0.72      0.72      0.72        32
weighted avg       0.72      0.72      0.72        32



# GraphSage MiniBatch Training
https://www-cs-faculty.stanford.edu/people/jure/pubs/graphsage-nips17.pdf

GraphSage uses neighborhood sampling at each epoch to give guarantees on the run time complexity during training. This is feasible by using reduced adjacency matrices for each minibatch that contain only $k$ sampled neighbors of the nodes in the minibatch.

In [5]:
def sample_neighbors(network, node, sample_size):
    pass